In [1]:
from pyiceberg.catalog import load_catalog
from pyiceberg.exceptions import TableAlreadyExistsError
from pyiceberg.schema import Schema
from pyiceberg.types import StringType, DoubleType, NestedField
import pandas as pd
from IPython.display import display
import pyarrow as pa

In [2]:
MINIO_URL = "http://172.18.0.4:9000"
MINIO_ACCESS_KEY = "minioadmin"
MINIO_SECRET_KEY = "minioadmin"

########################### REST Catalog  ###########################
RAW_CATALOG_URL = "http://172.18.0.3"
RAW_CATALOG_PORT = "8181"
RAW_CATALOG_NAME = "RAW"

TABLE_CATALOG_URL = "http://172.18.0.2"
TABLE_CATALOG_PORT = "8182"
TABLE_CATALOG_NAME = "TABLE"

# Charger le catalogue Iceberg RAW
catalogRAW = load_catalog('RAW', **{
    'uri': f"{RAW_CATALOG_URL}:{RAW_CATALOG_PORT}",
    's3.endpoint': MINIO_URL,
    's3.access-key-id': MINIO_ACCESS_KEY,
    's3.secret-access-key': MINIO_SECRET_KEY,
})

In [3]:
schema = Schema(
    NestedField(1, "city", StringType(), required=False),
    NestedField(2, "lat", DoubleType(), required=False),
    NestedField(3, "long", DoubleType(), required=False),
)

table_name = "my_namespace.cities_1"  # Specify your table name

try: 
    # Create the table
    my_table = catalogRAW.create_table(
        identifier=table_name,
        schema=schema,
    )
except TableAlreadyExistsError:
    print(f"Table {table_name} already exists in the catalog")

Table my_namespace.cities_1 already exists in the catalog


In [ ]:
df = pa.Table.from_pylist(
    [
        {"city": "Amsterdam", "lat": 52.371807, "long": 4.896029},
        {"city": "San Francisco", "lat": 37.773972, "long": -122.431297},
        {"city": "Drachten", "lat": 53.11254, "long": 6.0989},
        {"city": "Paris", "lat": 48.864716, "long": 2.349014},
    ],
)

In [ ]:
table = catalogRAW.load_table("my_namespace.cities_1")
table.overwrite(df)

In [4]:
scan = catalogRAW.load_table("my_namespace.cities_1").scan()
pandas_df = scan.to_pandas()
df = pd.DataFrame(pandas_df)
display(df)

,city,lat,long
0,Amsterdam,52.371807,4.896029
1,San Francisco,37.773972,-122.431297
2,Drachten,53.112540,6.098900
3,Paris,48.864716,2.349014
